# Actividad_2 — Big Data
**Alumno:** Justin Beckham Cardona Yepes 
**Curso:** Big Data  
**Fecha:** 23/11/2025

**Objetivo:** Buscar/recolectar un conjunto de datos y desplegarlo sobre una infraestructura virtual (Databricks CE / entorno PySpark), diseñando el esquema de almacenamiento, configurando la arquitectura básica, cargando datos desde Kaggle y validando el procesamiento con Spark y SQL.



## Índice
1. Diseño del esquema (DDL / StructType)  
2. Configuración y evidencia del entorno (nota: en Databricks CE no siempre se pueden crear clusters; aquí documentamos y mostramos información disponible)  
3. Ingesta de datos (Kaggle o archivo local) y creación de tabla  
4. Validaciones: metadatos, describe, SELECT, GROUP BY — en Spark y SQL  
5. Comparativa: Ventajas y desventajas SQL vs Spark  
6. Conclusiones y entrega (checklist)


## 1. Diseño del esquema (ER + diccionario de datos)

**Dataset elegido:** PlayStation Sales and Metadata (PS3, PS4, PS5) (Kaggle).

**Objetivo del modelo analítico:** normalizar el dataset para análisis de ventas por juego, plataforma, región y desarrolladora.

**Entidades principales:**
- `desarrolladora` — información de publishers/developers.
- `juego` — metadata por juego.
- `venta` — ventas por región/año.

A continuación se presenta el diccionario de datos y el DDL propuesto.


# Diccionario de Datos – Esquema Playstation

## Tabla: desarrolladora

| Campo               | Tipo   | Nulo | Descripción                                                   |
|---------------------|--------|------|----------------------------------------------------------------|
| id_desarrolladora   | INT    | NO   | Identificador único de la desarrolladora/publicadora.         |
| developer           | STRING | SÍ   | Nombre del estudio desarrollador.                             |
| publisher           | STRING | SÍ   | Empresa encargada de la publicación del juego.                |

---

## Tabla: juego

| Campo               | Tipo    | Nulo | Descripción                                                    |
|---------------------|---------|------|----------------------------------------------------------------|
| id_juego            | INT     | NO   | Identificador único del videojuego.                            |
| name                | STRING  | SÍ   | Nombre del videojuego.                                         |
| console             | STRING  | SÍ   | Consola de lanzamiento (PS1, PS2, PS3, etc.).                  |
| publisher           | STRING  | SÍ   | Publicador del juego según el dataset.                         |
| developer           | STRING  | SÍ   | Desarrollador del juego según el dataset.                      |
| genres              | STRING  | SÍ   | Género(s) del juego.                                           |
| rating              | DOUBLE  | SÍ   | Calificación o puntuación del juego.                           |
| id_desarrolladora   | INT     | SÍ   | Llave foránea hacia desarrolladora.id_desarrolladora.          |

---

## Tabla: venta

| Campo        | Tipo    | Nulo | Descripción                                     |
|--------------|---------|------|--------------------------------------------------|
| id_venta     | INT     | NO   | Identificador único de la venta.                 |
| id_juego     | INT     | NO   | FK hacia juego.id_juego.                         |
| total_sales  | DOUBLE  | SÍ   | Ventas totales (en millones de unidades).        |

---

# Relaciones



- **juego.id_desarrolladora** → desarrolladora.id_desarrolladora  
- **venta.id_juego** → juego.id_juego



In [0]:
%sql
-- Crear base de datos
CREATE DATABASE IF NOT EXISTS bigdata2025;
USE bigdata2025;

-- TABLA DESARROLLADORA
CREATE TABLE IF NOT EXISTS desarrolladora (
  id_desarrolladora INT,
  developer STRING,
  publisher STRING
)
USING DELTA;

-- TABLA JUEGO
CREATE TABLE IF NOT EXISTS juego (
  id_juego INT,
  name STRING,
  console STRING,
  publisher STRING,
  developer STRING,
  genres STRING,
  rating DOUBLE,
  id_desarrolladora INT
)
USING DELTA;

-- TABLA VENTA
CREATE TABLE IF NOT EXISTS venta (
  id_venta INT,
  id_juego INT,
  total_sales DOUBLE
)
USING DELTA;

![](/Volumes/workspace/bigdata2025/playstation_schema/playstation_schema.drawio.png)


# 2. Configuración y Evidencia de la Infraestructura en Databricks

## Investigación: Procedimiento de Creación y Configuración de Clúster
En un entorno profesional de Databricks (Standard, Premium o Enterprise), la configuración de la infraestructura es un paso crítico para garantizar el rendimiento y control de costos. A continuación, se describen los pasos teóricos para aprovisionar un clúster, contrastándolos con la versión Community Edition utilizada en esta práctica.

### Pasos para configurar un Clúster (All-Purpose Compute):
1.  **Navegación:** En la barra lateral izquierda, seleccionar **Compute** (Cómputo).
2.  **Creación:** Hacer clic en el botón azul **Create compute**.
3.  **Configuración General:**
    * **Databricks Runtime Version:** Se selecciona la versión de Spark y Scala. Para esta práctica se utiliza una versión LTS (Long Term Support) como la **13.3 LTS o 14.3 LTS (Scala 2.12, Spark 3.4/3.5)** para asegurar estabilidad.
    * **Access Mode:** Se define si el clúster es compartido (*Multi-user*) o exclusivo (*Single User*).
4.  **Dimensionamiento (Worker & Driver Types):**
    * Se eligen las instancias de máquina virtual (ej. AWS EC2 o Azure VMs).
    * **Workers:** Se define el rango de escalado automático (*Autoscaling*), por ejemplo, mín: 2, máx: 8 nodos.
    * **Driver:** Generalmente del mismo tipo que los workers.
5.  **Opciones Avanzadas:**
    * **Terminación automática:** Se configura un tiempo (ej. 45 min) para apagar el clúster si está inactivo y ahorrar costos.
    * **Variables de entorno:** Configuración de claves o parámetros de Spark específicos.

### Infraestructura en Databricks Community Edition (CE)
En esta versión gratuita, las opciones anteriores están limitadas por diseño:
* **Limitación:** Solo permite crear un clúster de **1 Driver y 0 Workers** (sin escalabilidad distribuida real).
* **Capacidad:** Aproximadamente 15 GB de RAM y 2 Núcleos.
* **Gestión:** No permite elegir tipo de instancia ni autoscaling.

### 📥 Carga manual del archivo a DBFS

1. Descargué el dataset desde Kaggle manualmente en formato CSV.  
2. En Databricks, ingresé a **"Data" → "Add Data" → "Upload File"**.  
3. Seleccioné el archivo *playstation.csv* y lo cargué a:
dbfs:/FileStore/playstation.csv

## Lectura del Dataset en Spark aplicando el esquema diseñado

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

schema_juego = StructType([
    StructField("id_juego", IntegerType(), False),
    StructField("name", StringType(), True),
    StructField("console", StringType(), True),
    StructField("publisher", StringType(), True),
    StructField("developer", StringType(), True),
    StructField("genres", StringType(), True),
    StructField("rating", StringType(), True)
])

schema_venta = StructType([
    StructField("id_venta", IntegerType(), False),
    StructField("id_juego", IntegerType(), False),
    StructField("total_sales", DoubleType(), True)
])

schema_desarrolladora = StructType([
    StructField("id_desarrolladora", IntegerType(), False),
    StructField("developer", StringType(), True),
    StructField("publisher", StringType(), True)
])

# 1. Cargar la tabla REAL del catálogo Workspace
df_raw = spark.table("workspace.bigdata2025.playstationdataset2025")

# 2. Mostrar datos
df_raw.show(5)
df_raw.printSchema()


+--------------------+-------+--------------------+--------------+--------------+-------------+-----------+---------+---------+-----------+-----------+------------+-----------+------+-------------+----------+---------------+--------------------+
|                Game|Console|                Name|     Publisher|     Developer|Total Shipped|Total Sales| NA Sales|PAL Sales|Japan Sales|Other Sales|Release Date|Last Update|rating|ratings_count|metacritic|         genres|           platforms|
+--------------------+-------+--------------------+--------------+--------------+-------------+-----------+---------+---------+-----------+-----------+------------+-----------+------+-------------+----------+---------------+--------------------+
|PS3_Grand Theft A...|    PS3|  Grand Theft Auto V|Rockstar Games|Rockstar North|          0.0| 20320000.0|6370000.0|9850000.0|   990000.0|  3120000.0|  17/09/2013|       NULL|  4.47|       7225.0|      92.0|         Action|PC, PlayStation 5...|
|PS4_Grand Theft

## Creación de las tablas en Spark (Delta Format, obligatorio)

### Crear base de datos:

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS bigdata2025;
USE bigdata2025;


### Crear tabla JUEGO (Delta)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS juego (
  id_juego INT,
  name STRING,
  console STRING,
  publisher STRING,
  developer STRING,
  genres STRING,
  rating STRING
)
USING delta;


### Crear tabla VENTA

In [0]:
%sql
CREATE TABLE IF NOT EXISTS venta (
  id_venta INT,
  id_juego INT,
  total_sales DOUBLE
)
USING delta;


### Crear tabla DESARROLLADORA

In [0]:
%sql
CREATE TABLE IF NOT EXISTS desarrolladora (
  id_desarrolladora INT,
  developer STRING,
  publisher STRING
)
USING delta;


###Cargar datos desde el DataFrame hacia las tablas

In [0]:
# 1. Cargar el dataframe original desde el catálogo
df_raw = spark.table("workspace.bigdata2025.playstationdataset2025")

df_raw.show(5)


from pyspark.sql.functions import monotonically_increasing_id

df_juego = df_raw.select(
    "Name",
    "Console",
    "genres"
)


df_juego.write.mode("overwrite").format("delta").saveAsTable("bigdata2025.juego")


+--------------------+-------+--------------------+--------------+--------------+-------------+-----------+---------+---------+-----------+-----------+------------+-----------+------+-------------+----------+---------------+--------------------+
|                Game|Console|                Name|     Publisher|     Developer|Total Shipped|Total Sales| NA Sales|PAL Sales|Japan Sales|Other Sales|Release Date|Last Update|rating|ratings_count|metacritic|         genres|           platforms|
+--------------------+-------+--------------------+--------------+--------------+-------------+-----------+---------+---------+-----------+-----------+------------+-----------+------+-------------+----------+---------------+--------------------+
|PS3_Grand Theft A...|    PS3|  Grand Theft Auto V|Rockstar Games|Rockstar North|          0.0| 20320000.0|6370000.0|9850000.0|   990000.0|  3120000.0|  17/09/2013|       NULL|  4.47|       7225.0|      92.0|         Action|PC, PlayStation 5...|
|PS4_Grand Theft

##3. Validaciones en Spark (PySpark)

##Metadatos


In [0]:
df_juego.printSchema()
df_juego.describe().show()

root
 |-- Name: string (nullable = true)
 |-- Console: string (nullable = true)
 |-- genres: string (nullable = true)

+-------+--------------------+----------+------------------+
|summary|                Name|   Console|            genres|
+-------+--------------------+----------+------------------+
|  count|                4946|      4946|              2795|
|   mean|                NULL|      NULL|             69.25|
| stddev|                NULL|      NULL|30.379543555930308|
|    min|      Delicious! 2!!| Elsewhere|             105.0|
|    max|uDraw Studio: Ins...|       PS5|  Strategy, Sports|
+-------+--------------------+----------+------------------+



##Conteos y muestras

In [0]:
df_juego.count()
df_juego.show(10)


+--------------------+-------+---------------+
|                Name|Console|         genres|
+--------------------+-------+---------------+
|  Grand Theft Auto V|    PS3|         Action|
|  Grand Theft Auto V|    PS4|         Action|
|Call of Duty: Bla...|    PS4|           NULL|
|Red Dead Redempti...|    PS4|         Action|
|Call of Duty: Bla...|    PS3|Shooter, Action|
|  Call of Duty: WWII|    PS4|Shooter, Action|
|Call of Duty: Mod...|    PS3|Shooter, Action|
|Call of Duty: Bla...|    PS3|Shooter, Action|
|             FIFA 18|    PS4|         Sports|
|             FIFA 17|    PS4|         Sports|
+--------------------+-------+---------------+
only showing top 10 rows


##GROUP BY en PySpark

In [0]:
df_juego.groupBy("console").count().show()

+--------------------+-----+
|             console|count|
+--------------------+-----+
|                 PS3| 1885|
|                 PS4| 1976|
|   000: Space Marine|    1|
|                NULL|   17|
|000: Inquisitor -...|    1|
|     000: Deathwatch|    1|
|                 PS5| 1074|
|   000: Rogue Trader|    1|
|000: Chaos Gate –...|    1|
|        000: Boltgun|    1|
| 000: Space Marine 2|    1|
|           Elsewhere|    1|
|            Princess|    1|
|               Serve|    2|
+--------------------+-----+



##4. Validaciones en SQL

##Metadatos con DESCRIBE TABLE

In [0]:
%sql
DESCRIBE TABLE bigdata2025.juego;

col_name,data_type,comment
id_juego,int,null
name,string,null
console,string,null
publisher,string,null
developer,string,null
genres,string,null
rating,double,null
id_desarrolladora,int,null


##Mostrar estructura completa


In [0]:
%sql
SHOW CREATE TABLE bigdata2025.juego;

createtab_stmt
"CREATE TABLE workspace.bigdata2025.juego ( id_juego INT, name STRING, console STRING, publisher STRING, developer STRING, genres STRING, rating DOUBLE, id_desarrolladora INT) USING delta TBLPROPERTIES ( 'delta.enableDeletionVectors' = 'true', 'delta.enableRowTracking' = 'true', 'delta.feature.appendOnly' = 'supported', 'delta.feature.deletionVectors' = 'supported', 'delta.feature.domainMetadata' = 'supported', 'delta.feature.invariants' = 'supported', 'delta.feature.rowTracking' = 'supported', 'delta.minReaderVersion' = '3', 'delta.minWriterVersion' = '7', 'delta.parquet.compression.codec' = 'zstd')"


##Consulta SELECT

In [0]:
%sql
SELECT name, console, rating FROM bigdata2025.juego LIMIT 10;

name,console,rating
Grand Theft Auto V,PS3,null
Grand Theft Auto V,PS4,null
Call of Duty: Black Ops 3,PS4,null
Red Dead Redemption 2,PS4,null
Call of Duty: Black Ops II,PS3,null
Call of Duty: WWII,PS4,null
Call of Duty: Modern Warfare 3,PS3,null
Call of Duty: Black Ops,PS3,null
FIFA 18,PS4,null
FIFA 17,PS4,null


##GROUP BY en SQL

In [0]:
%sql
SELECT console, COUNT(*) AS juegos_por_console
FROM bigdata2025.juego
GROUP BY console
ORDER BY juegos_por_console DESC;


console,juegos_por_console
PS4,1976
PS3,1885
PS5,1074
null,17
Serve,2
000: Space Marine 2,1
Elsewhere,1
000: Chaos Gate – Daemonhunters,1
000: Space Marine,1
000: Rogue Trader,1


##Conteo total

In [0]:
%sql
SELECT COUNT(*) FROM bigdata2025.juego;

COUNT(*)
4963


##5. Tabla comparativa: SQL vs PySpark (Markdown)

## 🔍 Comparación: SQL vs PySpark

| Característica | SQL (Spark SQL) | PySpark |
|----------------|-----------------|---------|
| Facilidad | Muy fácil y declarativo | Requiere más código |
| Curva de aprendizaje | Baja | Media–Alta |
| Escalabilidad | Alta (pero limitada a SQL) | Muy alta, diseñado para Big Data |
| Complejidad | Ideal para consultas simples | Ideal para ETL, ML, UDFs |
| Integración BI | Excelente | Limitada |
| Flexibilidad | Baja | Muy alta (APIs, MLlib, RDDs) |


## 🔵 Comparación entre SQL y Spark

A continuación se presenta una tabla comparativa clara y breve sobre las ventajas y desventajas de usar **SQL** y **Apache Spark** en procesos de análisis de datos y Big Data.

| Tecnología | Ventajas | Desventajas |
|-----------|----------|-------------|
| **SQL** | - Sintaxis sencilla y declarativa.<br>- Ideal para consultas rápidas y análisis descriptivo.<br>- Amplia integración con herramientas BI (Power BI, Tableau, etc.).<br>- Fácil de aprender y leer.<br>- Eficiente para datasets pequeños o medianos. | - No escala bien para Big Data real.<br>- Limitado para pipelines complejos.<br>- No está diseñado para procesamiento distribuido.<br>- Pocas capacidades para ML, UDFs o transformaciones avanzadas. |
| **Spark (PySpark / Spark SQL)** | - Diseñado para Big Data y procesamiento distribuido.<br>- Altamente escalable y tolerante a fallos.<br>- APIs potentes (DataFrames, MLlib, streaming).<br>- Excelente integración con Delta Lake y Databricks.<br>- Maneja grandes volúmenes de datos de forma eficiente. | - Curva de aprendizaje más alta.<br>- Requiere programación (Python/Scala) y conceptos de paralelismo.<br>- Menos intuitivo para usuarios principiantes.<br>- Sobrecoste para datasets muy pequeños. |

### 📌 Conclusión
- **SQL** es ideal para análisis rápidos y consultas directas.
- **Spark** se utiliza cuando los datos son demasiado grandes para un solo equipo o cuando se necesitan pipelines complejos y escalables.
